## Data/MC Comparison for Brem-Induced Clusters

#### Comparison of Clusters in Data vs MC. This is to validate the signal reconstruction process. We compare cluster from Z->MuMu events in Data vs those from a DY->ZMuMu (50-120 GeV for MLL). Data is from 2023D, and MC is from the postBPix, normalized to the appropriate value with an initial DBSCAN skim applied

In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import pandas as pd

Welcome to JupyROOT 6.28/00


In [2]:
sys.path.append("/uscms/home/amalbert/nobackup/CMSSW_14_1_0_pre4/src/RazorCommon/Tools/bin")
import importlib
import getMuonScaleFactor

In [3]:
#paths
#data_path = "/eos/uscms/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/Run2023B_skim_TnPOutput_MuonFilters_goodLumi.root"
#MC_path = "/uscms/home/amalbert/nobackup/CMSSW_10_6_20/src/run3_llp_analyzer/MCSummer23_DY50to120_postBPix_TnP_Output_DNNVariables_forDNN_617pb_weighted.root"
MC_path = "/eos/uscms/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/MC_Summer23/CSC_Comparison/MCSummer23_DYMLL50-120_TnPOutput_updateTagPt_18411pb_weighted.root"
#data_path = "/eos/uscms/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/MCSummer23_DY50to120_postBPix_newestTnP_Output_617pb_weighted.root"
data_path= "/eos/uscms/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/Data_2023_preBPix/CSC_Comparison/TnP_Ouput_preBPix_updateTagPt_goodLumi.root"

In [4]:
data_TFile = rt.TFile(data_path)
MC_TFile = rt.TFile(MC_path)

data_tree_unfiltered = data_TFile.Get("MuonSystem")
MC_tree_unfiltered = MC_TFile.Get("MuonSystem")

In [5]:
ak.behavior.update(candidate.behavior)

def getLZDF(f,nEvents=-1,version="new"): #lazy dataframe with events that have cluster matched to probe muon
    events_raw = uproot.open(f)['MuonSystem']
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    #events = events[start:stop]
    return events

In [6]:
events_data = getLZDF(data_path)
events_MC = getLZDF(MC_path)

In [7]:
events_data.fields

['runNum',
 'MC_condition',
 'lumiSec',
 'evtNum',
 'mH',
 'mX',
 'ctau',
 'category',
 'npv',
 'npu',
 'weight',
 'lepOverallSF',
 'pileupWeight',
 'pileupWeightUp',
 'pileupWeightDown',
 'numProbeMuons',
 'numTag',
 'Flag_HBHENoiseFilter',
 'Flag_BadPFMuonFilter',
 'Flag_BadPFMuonDzFilter',
 'Flag_HBHEIsoNoiseFilter',
 'Flag_CSCTightHaloFilter',
 'Flag_globalSuperTightHalo2016Filter',
 'Flag_goodVertices',
 'Flag_ecalBadCalibFilter',
 'Flag_BadChargedCandidateFilter',
 'Flag_eeBadScFilter',
 'Flag_hfNoisyHitsFilter',
 'Flag_all',
 'Flag2_HBHENoiseFilter',
 'Flag2_HBHEIsoNoiseFilter',
 'Flag2_BadPFMuonFilter',
 'Flag2_globalSuperTightHalo2016Filter',
 'Flag2_globalTightHalo2016Filter',
 'Flag2_BadChargedCandidateFilter',
 'Flag2_EcalDeadCellTriggerPrimitiveFilter',
 'Flag2_ecalBadCalibFilter',
 'Flag2_eeBadScFilter',
 'Flag2_all',
 'rho',
 'met',
 'metPhi',
 'gHiggsPt',
 'gHiggsE',
 'gHiggsEta',
 'gHiggsPhi',
 'nCscRings',
 'nDtRings',
 'nCscRechitClusters',
 'cscRechitCluster_match_g

In [8]:
#apply initial selections to events
mask_data = np.logical_and(events_data.ZMass>81.2, events_data.ZMass<101.2)
mask_data = np.logical_and(mask_data, ak.sum(events_data.lepTag, axis=1)==2)
mask_data = events_data.runNum>0
events_data = events_data[mask_data]

In [9]:
#apply initial selections to events
mask_MC = np.logical_and(events_MC.ZMass>81.2, events_MC.ZMass<101.2)
mask_MC = np.logical_and(mask_MC, ak.sum(events_MC.lepTag, axis=1)==2)
mask_MC = events_MC.runNum>0
events_MC = events_MC[mask_MC]
#events_MC = events_MC[~ak.is_none(events_MC)]

In [10]:
#for data
frac_s1_data = (events_data.cscRechitClusterNRechitChamberPlus11+events_data.cscRechitClusterNRechitChamberMinus11+events_data.cscRechitClusterNRechitChamberPlus12+events_data.cscRechitClusterNRechitChamberMinus12+events_data.cscRechitClusterNRechitChamberPlus13+events_data.cscRechitClusterNRechitChamberMinus13)/events_data.cscRechitClusterSize
frac_s2_data = (events_data.cscRechitClusterNRechitChamberPlus21+events_data.cscRechitClusterNRechitChamberMinus21+events_data.cscRechitClusterNRechitChamberPlus22+events_data.cscRechitClusterNRechitChamberMinus22)/events_data.cscRechitClusterSize
frac_s3_data = (events_data.cscRechitClusterNRechitChamberPlus31+events_data.cscRechitClusterNRechitChamberMinus31+events_data.cscRechitClusterNRechitChamberPlus32+events_data.cscRechitClusterNRechitChamberMinus32)/events_data.cscRechitClusterSize
frac_s4_data = (events_data.cscRechitClusterNRechitChamberPlus41+events_data.cscRechitClusterNRechitChamberMinus41+events_data.cscRechitClusterNRechitChamberPlus42+events_data.cscRechitClusterNRechitChamberMinus42)/events_data.cscRechitClusterSize

frac_rw1_data = (events_data.cscRechitClusterNRechitChamberPlus11+events_data.cscRechitClusterNRechitChamberMinus11+events_data.cscRechitClusterNRechitChamberPlus21+events_data.cscRechitClusterNRechitChamberMinus21+events_data.cscRechitClusterNRechitChamberPlus31+events_data.cscRechitClusterNRechitChamberMinus31+events_data.cscRechitClusterNRechitChamberPlus41+events_data.cscRechitClusterNRechitChamberMinus41)/events_data.cscRechitClusterSize
frac_rw2_data = (events_data.cscRechitClusterNRechitChamberPlus12+events_data.cscRechitClusterNRechitChamberMinus12+events_data.cscRechitClusterNRechitChamberPlus22+events_data.cscRechitClusterNRechitChamberMinus22+events_data.cscRechitClusterNRechitChamberPlus32+events_data.cscRechitClusterNRechitChamberMinus32+events_data.cscRechitClusterNRechitChamberPlus42+events_data.cscRechitClusterNRechitChamberMinus42)/events_data.cscRechitClusterSize
frac_rw3_data = (events_data.cscRechitClusterNRechitChamberPlus13+events_data.cscRechitClusterNRechitChamberMinus13)/events_data.cscRechitClusterSize

In [11]:
#for data
frac_s1_MC = (events_MC.cscRechitClusterNRechitChamberPlus11+events_MC.cscRechitClusterNRechitChamberMinus11+events_MC.cscRechitClusterNRechitChamberPlus12+events_MC.cscRechitClusterNRechitChamberMinus12+events_MC.cscRechitClusterNRechitChamberPlus13+events_MC.cscRechitClusterNRechitChamberMinus13)/events_MC.cscRechitClusterSize
frac_s2_MC = (events_MC.cscRechitClusterNRechitChamberPlus21+events_MC.cscRechitClusterNRechitChamberMinus21+events_MC.cscRechitClusterNRechitChamberPlus22+events_MC.cscRechitClusterNRechitChamberMinus22)/events_MC.cscRechitClusterSize
frac_s3_MC = (events_MC.cscRechitClusterNRechitChamberPlus31+events_MC.cscRechitClusterNRechitChamberMinus31+events_MC.cscRechitClusterNRechitChamberPlus32+events_MC.cscRechitClusterNRechitChamberMinus32)/events_MC.cscRechitClusterSize
frac_s4_MC = (events_MC.cscRechitClusterNRechitChamberPlus41+events_MC.cscRechitClusterNRechitChamberMinus41+events_MC.cscRechitClusterNRechitChamberPlus42+events_MC.cscRechitClusterNRechitChamberMinus42)/events_MC.cscRechitClusterSize

frac_rw1_MC = (events_MC.cscRechitClusterNRechitChamberPlus11+events_MC.cscRechitClusterNRechitChamberMinus11+events_MC.cscRechitClusterNRechitChamberPlus21+events_MC.cscRechitClusterNRechitChamberMinus21+events_MC.cscRechitClusterNRechitChamberPlus31+events_MC.cscRechitClusterNRechitChamberMinus31+events_MC.cscRechitClusterNRechitChamberPlus41+events_MC.cscRechitClusterNRechitChamberMinus41)/events_MC.cscRechitClusterSize
frac_rw2_MC = (events_MC.cscRechitClusterNRechitChamberPlus12+events_MC.cscRechitClusterNRechitChamberMinus12+events_MC.cscRechitClusterNRechitChamberPlus22+events_MC.cscRechitClusterNRechitChamberMinus22+events_MC.cscRechitClusterNRechitChamberPlus32+events_MC.cscRechitClusterNRechitChamberMinus32+events_MC.cscRechitClusterNRechitChamberPlus42+events_MC.cscRechitClusterNRechitChamberMinus42)/events_MC.cscRechitClusterSize
frac_rw3_MC = (events_MC.cscRechitClusterNRechitChamberPlus13+events_MC.cscRechitClusterNRechitChamberMinus13)/events_MC.cscRechitClusterSize

In [12]:
data_tree = data_tree_unfiltered.CloneTree(0)
data_tree.SetMaxTreeSize(100000000);
#data_tree.SetAutoSave(3000000)
data_tree_unfiltered.AddFriend(data_tree)
for idx, entry in enumerate(data_tree_unfiltered):
    if mask_data[idx]:
        data_tree.Fill()



Error in <TBranch::WriteBasketImpl>: basket's WriteBuffer failed.
Error in <TBranch::TBranch::Fill>: Failed to write out basket.

Error in <TTree::Fill>: Failed filling branch:MuonSystem.cscRechitClusterDNN_bkgMC_plusBeamHalo, nbytes=-1, entry=265
 This error is symptomatic of a Tree created as a memory-resident Tree
 Instead of doing:
    TTree *T = new TTree(...)
    TFile *f = new TFile(...)
 you should do:
    TFile *f = new TFile(...)
    TTree *T = new TTree(...)


Error in <TBranch::WriteBasketImpl>: basket's WriteBuffer failed.
Error in <TBranch::TBranch::Fill>: Failed to write out basket.

Error in <TTree::Fill>: Failed filling branch:MuonSystem.cscRechitClusterDNN_bkgMC_plusBeamHalo, nbytes=-1, entry=530
 This error is symptomatic of a Tree created as a memory-resident Tree
 Instead of doing:
    TTree *T = new TTree(...)
    TFile *f = new TFile(...)
 you should do:
    TFile *f = new TFile(...)
    TTree *T = new TTree(...)


Error in <TBranch::WriteBasketImpl>: basket's Wr

In [13]:

MC_tree = MC_tree_unfiltered.CloneTree(0)
MC_tree.SetAutoSave(30000000);
MC_tree_unfiltered.AddFriend(MC_tree)
for idx, entry in enumerate(MC_tree_unfiltered):
    if mask_MC[idx]:
        MC_tree.Fill()

Error in <TBranch::WriteBasketImpl>: basket's WriteBuffer failed.
Error in <TBranch::TBranch::Fill>: Failed to write out basket.

Error in <TTree::Fill>: Failed filling branch:MuonSystem.cscRechitClusterDNN_bkgMC_plusBeamHalo, nbytes=-1, entry=265
 This error is symptomatic of a Tree created as a memory-resident Tree
 Instead of doing:
    TTree *T = new TTree(...)
    TFile *f = new TFile(...)
 you should do:
    TFile *f = new TFile(...)
    TTree *T = new TTree(...)


Error in <TBranch::WriteBasketImpl>: basket's WriteBuffer failed.
Error in <TBranch::TBranch::Fill>: Failed to write out basket.

Error in <TTree::Fill>: Failed filling branch:MuonSystem.cscRechitClusterDNN_bkgMC_plusBeamHalo, nbytes=-1, entry=530
 This error is symptomatic of a Tree created as a memory-resident Tree
 Instead of doing:
    TTree *T = new TTree(...)
    TFile *f = new TFile(...)
 you should do:
    TFile *f = new TFile(...)
    TTree *T = new TTree(...)


Error in <TBranch::WriteBasketImpl>: basket's Wr

In [14]:

def makeEventMask(events, noMaskList: list=[], noCuts=False):
    mask = ak.flatten(events.cscRechitCluster_matchToProbeMuon)
#     phiMask = (((ak.flatten(events.cscRechitClusterPhi)>0.4) & (ak.flatten(events.cscRechitClusterPhi)<0.8))|(np.abs(ak.flatten(events.cscRechitClusterPhi))>2.8))
    #ZMask = ak.flatten(events.cscRechitClusterZ)<0
#     EtaMask = ak.flatten(events.cscRechitClusterEta)<-1.9
#     noise_mask = np.logical_and(phiMask, ZMask)
#     noise_mask = np.logical_and(noise_mask, EtaMask)
    #noise_mask = abs(ak.flatten(events.cscRechitClusterEta))>1.9
    #mask = ak.mask(mask, np.logical_not(noise_mask))
    #mask = ak.mask(mask, ak.flatten(events.cscRechitClusterMuonVetoPt>50))
    #if events.weight[0]==0:
#   #      #print("app cut")
    #    mask = ak.mask(mask, ak.flatten(frac_s2_data)>0.05)
    #else:
    #    mask = ak.mask(mask, ak.flatten(frac_s2_MC)>0.05)
    if "forward_veto" not in noMaskList:
        mask = ak.mask(mask, ak.flatten(events.cscRechitCluster_passME1112Veto))
    if noCuts:
        return mask
    if "timespread_veto" not in noMaskList:
        mask = ak.mask(mask, ak.flatten(events.cscRechitClusterTimeSpreadWeightedAll)<20)
    if "time_veto" not in noMaskList:
        mask = ak.mask(mask, ak.flatten(events.cscRechitClusterTimeWeighted)<12.5)
        mask = ak.mask(mask, ak.flatten(events.cscRechitClusterTimeWeighted)>-5)
    if "DNN_veto" not in noMaskList:
        remove9999 = ak.mask(ak.flatten(events.cscRechitClusterDNN_bkgMC_plusBeamHalo), ak.flatten(events.cscRechitClusterDNN_bkgMC_plusBeamHalo)>0)
        remove9999 = remove9999[~ak.is_none(remove9999)]
        DNNMask = remove9999>0.96
        mask = ak.mask(mask, DNNMask)
    if "clusterSize_veto" not in noMaskList:
        mask = ak.mask(mask, ak.flatten(events.cscRechitClusterSize)>160)
    
    return mask

In [15]:
importlib.reload(getMuonScaleFactor)

#loose ID
#data_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_BPix(np.array(events_data.lepPt), np.array(events_data.lepEta))
MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preBPix(np.array(events_MC.lepPt), np.array(events_MC.lepEta))

#looseISO
#data_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_BPix(np.array(events_data.lepPt), np.array(events_data.lepEta))
MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preBPix(np.array(events_MC.lepPt), np.array(events_MC.lepEta))

#tightID
#data_SF_TightID = getMuonScaleFactor.getTightIDEffArr_BPix(np.array(events_data.lepPt), np.array(events_data.lepEta))
MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preBPix(np.array(events_MC.lepPt), np.array(events_MC.lepEta))

#tightISO
#data_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_BPix(np.array(events_data.lepPt), np.array(events_data.lepEta))
MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preBPix(np.array(events_MC.lepPt), np.array(events_MC.lepEta))

#HLT
#data_SF_HLT = getMuonScaleFactor.getHLTEffArr_BPix(np.array(events_data.lepPt), np.array(events_data.lepEta))
MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preBPix(np.array(events_MC.lepPt), np.array(events_MC.lepEta))


In [16]:
def computeMCEfficiency(MCTree, arrayMC, cutsList, noCuts=False):
    sumOfWeights=0
    mask = makeEventMask(arrayMC, cutsList, noCuts)
    #print(ak.count_nonzero(mask[~ak.is_none(mask)]))
    clusterIndex=-1
    for evtNum, entry in enumerate(MCTree):

        for i in range(MCTree.nCscRechitClusters):
            
            clusterIndex+=1
            if mask[clusterIndex]:
                if MCTree.cscRechitCluster_matchToMuon1[i]==1:
                    probeIndex=0; tagIndex=1
                else:
                    probeIndex=1; tagIndex=0
                weight = getattr(MCTree, "weight")*getattr(MCTree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
                #weight = MC_SF_LooseID[evtNum][probeIndex]
                if MCTree.lepPt[tagIndex]>26:
                    weight=weight*MC_SF_HLT[evtNum][tagIndex]
                #print(weight)
                sumOfWeights+=weight
    return sumOfWeights

In [17]:
# data efficiencies
oneClusterData = ak.mask(events_data, events_data.nCscRechitClusters==1)
oneClusterData = oneClusterData[~ak.is_none(oneClusterData)]
num = ak.count_nonzero(makeEventMask(events_data))
print("Data Numerator: ", num)

denom_forward = ak.count_nonzero(makeEventMask(events_data, ['forward_veto']))
print("Forward Veto Efficiency: ", num/denom_forward*100)

denom_timespread = ak.count_nonzero(makeEventMask(events_data, ['timespread_veto']))
print("Timespread Veto Efficiency: ", num/denom_timespread*100)

denom_time = ak.count_nonzero(makeEventMask(events_data, ['time_veto']))
print("Time Veto Efficiency: ", num/denom_time*100)

denom_DNN = ak.count_nonzero(makeEventMask(events_data, ['DNN_veto']))
print("DNN Veto Efficiency: ", num/denom_DNN*100)

denom_clusterSize = ak.count_nonzero(makeEventMask(events_data, ['clusterSize_veto', 'timespread_veto']))
print("ClusterSize Veto Efficiency: ", num/denom_clusterSize*100)

Data Numerator:  660
Forward Veto Efficiency:  91.41274238227147
Timespread Veto Efficiency:  83.33333333333334
Time Veto Efficiency:  98.80239520958084
DNN Veto Efficiency:  37.909247558874206
ClusterSize Veto Efficiency:  12.533232054690469


In [18]:
# data efficiencies
print("computing other efficiency definition")
oneClusterData = ak.mask(events_data, events_data.nCscRechitClusters==1)
oneClusterData = oneClusterData[~ak.is_none(oneClusterData)]
denom = ak.count_nonzero(makeEventMask(events_data, [], True))
print("Data Denominator: ", denom)

#num_forward = ak.count_nonzero(makeEventMask(events_data, ['forward_veto']))
#print("Forward Veto Efficiency: ", num/denom_forward*100)

num_timespread = ak.count_nonzero(makeEventMask(events_data, ['clusterSize_veto','DNN_veto','time_veto']))
print("Timespread Veto Efficiency: ", num_timespread/denom*100)

num_time = ak.count_nonzero(makeEventMask(events_data, ['clusterSize_veto','DNN_veto','timespread_veto']))
print("Time Veto Efficiency: ", num_time/denom*100)

num_DNN = ak.count_nonzero(makeEventMask(events_data, ['timespread_veto','clusterSize_veto','time_veto']))
print("DNN Veto Efficiency: ", num_DNN/denom*100)

num_clusterSize = ak.count_nonzero(makeEventMask(events_data, ['timespread_veto','DNN_veto','time_veto']))
print("ClusterSize Veto Efficiency: ", num_clusterSize/denom*100)

computing other efficiency definition
Data Denominator:  37162
Timespread Veto Efficiency:  77.22135514773156
Time Veto Efficiency:  98.03024595016414
DNN Veto Efficiency:  14.635918411280342
ClusterSize Veto Efficiency:  5.669770195360853


In [19]:
num = computeMCEfficiency(MC_tree, events_MC, [])
print("MC Numerator: ", num)

#num_forward = ak.count_nonzero(makeEventMask(events_data, ['forward_veto']))
#print("Forward Veto Efficiency: ", num/denom_forward*100)

denom_timespread = computeMCEfficiency(MC_tree, events_MC, ['timespread_veto'])
print("Timespread Veto Efficiency: ", num/denom_timespread*100)

denom_time =computeMCEfficiency(MC_tree, events_MC, ['time_veto'])
print("Time Veto Efficiency: ", num/denom_time*100)

denom_DNN = computeMCEfficiency(MC_tree, events_MC, ['DNN_veto'])
print("DNN Veto Efficiency: ", num/denom_DNN*100)

denom_clusterSize = computeMCEfficiency(MC_tree, events_MC, ['clusterSize_veto'])
print("ClusterSize Veto Efficiency: ", num/denom_clusterSize*100)

MC Numerator:  1549.763003194928
Timespread Veto Efficiency:  99.38933567130053
Time Veto Efficiency:  99.44345689723274
DNN Veto Efficiency:  54.61136269679299
ClusterSize Veto Efficiency:  23.102348989083914


In [20]:
denom = computeMCEfficiency(MC_tree, events_MC, [], True)
print("MC Denominator: ", denom)

#num_forward = ak.count_nonzero(makeEventMask(events_data, ['forward_veto']))
#print("Forward Veto Efficiency: ", num/denom_forward*100)

num_timespread = computeMCEfficiency(MC_tree, events_MC, ['clusterSize_veto','DNN_veto','time_veto'])
print("Timespread Veto Efficiency: ", num_timespread/denom*100)

num_time =computeMCEfficiency(MC_tree, events_MC, ['clusterSize_veto','DNN_veto','timespread_veto'])
print("Time Veto Efficiency: ", num_time/denom*100)

num_DNN = computeMCEfficiency(MC_tree, events_MC, ['timespread_veto','clusterSize_veto','time_veto'])
print("DNN Veto Efficiency: ", num_DNN/denom*100)

num_clusterSize = computeMCEfficiency(MC_tree, events_MC, ['timespread_veto','DNN_veto','time_veto'])
print("ClusterSize Veto Efficiency: ", num_clusterSize/denom*100)

MC Denominator:  39463.83590524085
Timespread Veto Efficiency:  97.9613874554076
Time Veto Efficiency:  99.36933991986602
DNN Veto Efficiency:  18.07099415940617
ClusterSize Veto Efficiency:  7.238163642127062


In [21]:
events_MC.weight

<Array [6.92, 6.92, 6.92, ... 6.92, 6.92, 6.92] type='6483 * float32'>

In [22]:
MC_tree.GetEntries()

6483

### Make Appropriately-Style Plots

In [23]:
rt.gStyle.SetOptStat(0)
def make_ratio_plot(h_list_in, title = "", label = "", fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1]):
    h_list = []
    if in_tags == None:
        tag = []
    else:
        tag = in_tags
    for i, h in enumerate(h_list_in):
        h_list.append(h.Clone('h{}aux{}'.format(i, label)))
        if in_tags == None:
            tag.append(h.GetTitle())
    #print("tags: ", tag)
    c_out = rt.TCanvas("c_out_ratio"+label, "c_out_ratio"+label, 800, 800)
    pad1 = rt.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
    pad1.SetBottomMargin(0.03)
    pad1.SetLeftMargin(0.15)
    pad1.SetRightMargin(0.04)# pad2.SetGrid()
    if logy:
        pad1.SetLogy()

    pad1.Draw()
    pad1.cd()

    leg = rt.TLegend(0.5, 0.65, 0.9, 0.92)
    leg = rt.TLegend(0.7, 0.65, 0.9, 0.92)

    #leg = rt.TLegend(0.2, 0.7, 0.5, 0.9)
    # leg = rt.TLegend(0.7, 0.2, 0.9, 0.4)
    leg.SetBorderSize(0)
    leg.SetTextSize(0.045)
    leg.SetFillStyle(0)
    c_out.cd(1)

    scaled_h_list = []
    if scale:
        for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #h = h_unscaled.Scale(1/scales[i])
            #scaled_h_list.append(h_unscaled.Clone())
            h_unscaled.Scale(1/scales[i])
            scaled_h_list.append(h_unscaled)
    else:
        #for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #scaled_h_list.append(h)
        scaled_h_list = h_list
    for i, h in enumerate(scaled_h_list):
        h.GetXaxis().SetLabelSize(0)
        h.GetXaxis().SetTitle(label)
        h.GetYaxis().SetRangeUser(0, 1.1*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and not scale:
            h.GetYaxis().SetRangeUser(10e-2, 2*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and scale:
            h.GetYaxis().SetRangeUser(10e-4, 1)
        h.GetYaxis().SetTitleOffset(1.0)
        h.GetYaxis().SetTitleSize(0.06)
        h.GetYaxis().SetLabelSize(0.05)
        
        if scale:
            y_title = "Fraction of Events"
        else:
            y_title = "Events"
        
        h.GetYaxis().SetTitle()
        h.SetTitle(f"{title};adsf;{y_title}")
        #if ratio_index == 0:h.DrawCopy("hist")
        '''
        h.SetFillColor(h_list_in[i].GetLineColor())
        h.SetFillStyle(3002)
        #h.SetStats(1)
        h.SetLineColor(h_list_in[i].GetLineColor())
        h.SetLineWidth(2)
        h.SetMarkerColor(h_list_in[i].GetLineColor())
        h.SetMarkerSize(2)
        # if ratio_index == 0:
        #     # h.DrawCopy("hist")
        #     h.DrawCopy(draw_opt[i]+'same')
        # else:h.DrawCopy(draw_opt[i])
        #if ratio_index == 0 :h.DrawCopy(draw_opt[i]+"same")
        #h.DrawCopy("E2 HIST")
        '''
        if i==0:
            h.SetLineWidth(4)
            h.DrawCopy("hist")
            #h.SetFillStyle(0)
            h.SetFillColor(h_list_in[i].GetLineColor())
            h.SetFillStyle(3002)
            #h.SetStats(1)
            h.SetLineColor(h_list_in[i].GetLineColor())
            h.SetLineWidth(2)
            h.SetMarkerColor(h_list_in[i].GetLineColor())
            h.SetMarkerSize(2)
            h.DrawCopy(draw_opt[i] + "same")
            #h.SetFillStyle(0)
        else:
            h.SetLineWidth(2)
            h.DrawCopy(draw_opt[i] + "same")
        #else:h.DrawCopy(draw_opt[i])
        if len(text)>0:
            l = rt.TLatex()
            l.SetTextSize(0.045)
            if logy:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()/10, text)
            else:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()*0.8, text)
        #if i==1:
            #h.DrawCopy(draw_opt[i]+"same")
       #     h.Draw("E1 same")

        leg.AddEntry(h, tag[i], "lep")
    leg.Draw("same")

    c_out.cd()
    pad2 = rt.TPad("pad2", "pad2", 0, 0, 1, 0.3)
    pad2.SetTopMargin(0.03)
    pad2.SetBottomMargin(0.25)
    pad2.SetLeftMargin(0.15)
    pad2.SetRightMargin(0.04)# pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    band = scaled_h_list[ratio_index].Clone('h_band')
    for j in range(band.GetXaxis().GetNbins()):
        band.SetBinContent(j+1, 1.0)
        if h_list[ratio_index].GetBinContent(j+1) == 0:
            band.SetBinError(j+1, 0.0)
        else:
            band.SetBinError(j+1, scaled_h_list[ratio_index].GetBinError(j+1)/scaled_h_list[ratio_index].GetBinContent(j+1))
            #print(j, h_list_in[0].GetBinError(j+1)/h_list_in[0].GetBinContent(j+1))
    band.SetFillColor(scaled_h_list[ratio_index].GetLineColor())

    band.SetFillStyle(3002)
    band.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    #band.SetFillColorAlpha(0,0)
    #band.SetLineColor(0)
    
    band.GetYaxis().SetTitleOffset(0.5)
    band.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
    band.GetYaxis().SetTitleSize(0.11)
    band.GetYaxis().SetLabelSize(0.12)
    band.GetYaxis().SetNdivisions(506)
    band.GetXaxis().SetTitleOffset(0.95)
    band.GetXaxis().SetTitleSize(0.12)
    band.GetXaxis().SetLabelSize(0.12)
    band.GetXaxis().SetTickSize(0.07)
    
    band.SetYTitle('Ratio with {}'.format(tag[ratio_index]))
    band.SetXTitle(label)
    band.SetTitle("")
    band.DrawCopy('E2')
    ln = rt.TLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
    ln.SetLineWidth(3)
    ln.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    ln.DrawLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
     
    #print(ratio_index)
    for i, h in enumerate(scaled_h_list):
        if i == ratio_index:
            continue
        else:
            if fit:h.GetFunction("expo")
            h.Divide(scaled_h_list[ratio_index])
            # h.GetYaxis().SetTitleOffset(0.6)
            # h.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
            # h.GetYaxis().SetTitleSize(0.12)
            # h.GetYaxis().SetLabelSize(0.12)
            # h.GetYaxis().SetNdivisions(506)
            # h.GetXaxis().SetTitleOffset(0.95)
            # h.GetXaxis().SetTitleSize(0.12)
            # h.GetXaxis().SetLabelSize(0.12)
            # h.GetXaxis().SetTickSize(0.07)
            # h.SetYTitle('Ratio with {}'.format(tag[0]))
            # h.SetTitle("")
            #set relative error of ratio to be the relative error of data
            for j in range(h.GetXaxis().GetNbins()):
                if h_list[i].GetBinContent(j+1) == 0:
                    h.SetBinError(j+1, 0.0)
                else:
                    h.SetBinError(j+1, h_list_in[i].GetBinError(j+1)/h_list_in[i].GetBinContent(j+1)*h.GetBinContent(j+1))
            h.Draw('same'+draw_opt[i])
    
    pad2.Update()
    
    c_out.pad1 = pad1
    c_out.pad2 = pad2
    c_out.h_list = h_list
    c_out.leg = leg
    
    return c_out

### Compute MC Scale Factors

### Compute Hit Fraction Variables (DNN input variable)

### Make Data/MC Plots

In [24]:
import os
plot_info = {}
plot_output = "Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange"
os.makedirs(plot_output, exist_ok=True)

In [25]:
branch = "ZMass"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=80, xlow = 0, xup=150)
#data_LeadingPt.Project("LeadingPt_Data", "LeadingPt")
mask = makeEventMask(events_data)
clusterIndex=-1

for entry in data_tree:
    #if clusterIndex>=
    plottedCluster=False
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        #print(clusterIndex)
        if mask[clusterIndex] and not plottedCluster:
            data.Fill(getattr(data_tree, branch))
            plottedCluster=True
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=80, xlow = 0, xup=150)
mask = makeEventMask(events_MC)
clusterIndex=-1
sumOfWeights=0
for evtNum, entry in enumerate(MC_tree):
    plottedCluster=False
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex] and not plottedCluster:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            MC.Fill(getattr(MC_tree, branch), weight)
            plottedCluster=True
            sumOfWeights+=weight
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[branch] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Z Candidate Mass", "label": "Z Candidate Mass [GeV]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

In [26]:
branch = "ZMass"
name = branch+"_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=80, xlow = 0, xup=150)
#data_LeadingPt.Project("LeadingPt_Data", "LeadingPt")
mask = makeEventMask(events_data, [], True)
clusterIndex=-1

for entry in data_tree:
    #if clusterIndex>=
    plottedCluster=False
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        #print(clusterIndex)
        if mask[clusterIndex] and not plottedCluster:
            data.Fill(getattr(data_tree, branch))
            plottedCluster=True
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=80, xlow = 0, xup=150)
mask = makeEventMask(events_MC, [], True)
clusterIndex=-1
sumOfWeights=0
for evtNum, entry in enumerate(MC_tree):
    plottedCluster=False
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex] and not plottedCluster:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            MC.Fill(getattr(MC_tree, branch), weight)
            plottedCluster=True
            sumOfWeights+=weight
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[branch] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Z Candidate Mass", "label": "Z Candidate Mass [GeV]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [27]:
branch = "cscRechitClusterMuonVetoPt"
name = "MuonVetoPt"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=80, xlow = 0, xup=120)
mask = makeEventMask(events_data, [])
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=80, xlow = 0, xup=120)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
num=sumOfWeights   
print("numerator: ", num)
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Muon Veto Pt", "label": "Muon Veto Pt [GeV]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

numerator:  1549.763003194928


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [28]:
branch = "cscRechitClusterMuonVetoPt"
name = "MuonVetoPt_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=80, xlow = 0, xup=120)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=80, xlow = 0, xup=120)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
#num=sumOfWeights   
#print("numerator: ", num)
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Muon Veto Pt", "label": "Muon Veto Pt [GeV]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [29]:
branch = "cscRechitClusterDNN_bkgMC_plusBeamHalo"
name = "DNNScore"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=50, xlow = 0.5, xup=1.05)
mask = makeEventMask(events_data, ['DNN_veto'])
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ['DNN_veto'])
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=50, xlow = 0.5, xup=1.05)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "DNN Score", "label": "DNN Score", "scales": [sumOfWeights, data.GetEntries()], "logy": True}

cscRechitClusterDNN_bkgMC_plusBeamHalo efficiency  =  0.5461136269679299


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [30]:
branch = "cscRechitClusterDNN_bkgMC_plusBeamHalo"
name = "DNNScore_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=50, xlow = 0.5, xup=1.05)
mask = makeEventMask(events_data, ["DNN_veto"], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ["DNN_veto"], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=50, xlow = 0.5, xup=1.05)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            #if MC_tree.lepPt[tagIndex]>26:
            #    weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "DNN Score", "label": "DNN Score", "scales": [sumOfWeights, data.GetEntries()], "logy": True}

cscRechitClusterDNN_bkgMC_plusBeamHalo efficiency  =  0.03829819454074526


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [31]:
branch = "cscRechitClusterSize"
name = "clusterSize"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=40, xlow = 0, xup=400)
mask = makeEventMask(events_data, ['clusterSize_veto'])
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ['clusterSize_veto'])
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=40, xlow = 0, xup=400)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
        
MC.SetLineColor(rt.kRed)
print(branch + " efficiency  = ", num/sumOfWeights)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Size", "label": "N_{rechits}", "scales": [sumOfWeights, data.GetEntries()], "logy": True}

cscRechitClusterSize efficiency  =  0.23102348989083915


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [32]:
branch = "cscRechitClusterSize"
name = "clusterSize_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=40, xlow = 0, xup=400)
mask = makeEventMask(events_data, ['clusterSize_veto'], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ['clusterSize_veto'], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=40, xlow = 0, xup=400)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
        
MC.SetLineColor(rt.kRed)
print(branch + " efficiency  = ", num/sumOfWeights)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Size", "label": "N_{rechits}", "scales": [sumOfWeights, data.GetEntries()], "logy": True}

cscRechitClusterSize efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [33]:
branch = "cscRechitClusterEta"
name = "clusterEta"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -4, xup=4)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -4, xup=4)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Eta", "label": "Cluster Eta", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterEta efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [34]:
branch = "cscRechitClusterEta"
name = "clusterEta_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -4, xup=4)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -4, xup=4)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Eta", "label": "Cluster Eta", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterEta efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [35]:
branch = "cscRechitClusterPhi"
name = "clusterPhi"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Phi", "label": "Cluster Phi", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterPhi efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [36]:
branch = "cscRechitClusterPhi"
name = "clusterPhi_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Phi", "label": "Cluster Phi", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterPhi efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [37]:
branch = "cscRechitClusterTimeWeighted"
name = "clusterTime"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -8, xup=20)
mask = makeEventMask(events_data, ['time_veto'])
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ['time_veto'])
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -8, xup=20)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)       
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Weighted Cluster Time", "label": "Weighted Cluster Time [ns]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterTimeWeighted efficiency  =  0.9944345689723274


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [38]:
branch = "cscRechitClusterTimeWeighted"
name = "clusterTime_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -8, xup=20)
mask = makeEventMask(events_data, ['time_veto'], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ['time_veto'], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -8, xup=20)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)       
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Weighted Cluster Time", "label": "Weighted Cluster Time [ns]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterTimeWeighted efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [39]:
branch = "cscRechitClusterTimeSpreadWeightedAll"
name = "clusterTimeSpreadWeightedAll"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=50, xlow = -1, xup=40)
mask = makeEventMask(events_data, ["timespread_veto"])
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, ["timespread_veto"])
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=50, xlow = -1, xup=40)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Weighted Cluster Time Spread", "label": "Weighted Cluster Time Spread [ns]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterTimeSpreadWeightedAll efficiency  =  0.9938933567130053


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [40]:
branch = "cscRechitClusterTimeSpreadWeightedAll"
name = "clusterTimeSpreadWeightedAll_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=50, xlow = -1, xup=40)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=50, xlow = -1, xup=40)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Weighted Cluster Time Spread", "label": "Weighted Cluster Time Spread [ns]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterTimeSpreadWeightedAll efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [41]:
branch = "cscRechitClusterNStation10"
name = "clusterNStation10"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=5, xlow = 0, xup=5)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=5, xlow = 0, xup=5)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Number of Stations with >=10 Rechits", "label": "NStation10", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterNStation10 efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [42]:
branch = "cscRechitClusterNStation10"
name = "clusterNStation10_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=5, xlow = 0, xup=5)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=5, xlow = 0, xup=5)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Number of Stations with >=10 Rechits", "label": "NStation10", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterNStation10 efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [43]:
branch = "cscRechitClusterXSpread"
name = "clusterXSpread"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster X Spread", "label": "Cluster X Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterXSpread efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [44]:
branch = "cscRechitClusterXSpread"
name = "clusterXSpread_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster X Spread", "label": "Cluster X Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterXSpread efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [45]:
branch = "cscRechitClusterYSpread"
name = "clusterYSpread"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Y Spread", "label": "Cluster Y Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterYSpread efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [46]:
branch = "cscRechitClusterYSpread"
name = "clusterYSpread_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Y Spread", "label": "Cluster Y Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterYSpread efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [47]:
branch = "cscRechitClusterZSpread"
name = "clusterZSpread"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=200)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=200)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Z Spread", "label": "Cluster Z Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterZSpread efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [48]:
branch = "cscRechitClusterZSpread"
name = "clusterZSpread_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=200)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=200)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Z Spread", "label": "Cluster Z Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterZSpread efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [49]:
branch = "cscRechitClusterXYSpread"
name = "clusterXYSpread"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster XY Spread", "label": "Cluster XY Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterXYSpread efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [50]:
branch = "cscRechitClusterRSpread"
name = "clusterRSpread"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster R Spread", "label": "Cluster R Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterRSpread efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [51]:
branch = "cscRechitClusterRSpread"
name = "clusterRSpread_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster R Spread", "label": "Cluster R Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterRSpread efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [52]:
branch = "cscRechitClusterXYSpread"
name = "clusterXYSpread_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -5, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster XY Spread", "label": "Cluster XY Spread [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterXYSpread efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [53]:
branch = "cscRechitClusterSkewX"
name = "clusterSkewX"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -150, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew X", "label": "Cluster X Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewX efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [54]:
branch = "cscRechitClusterSkewX"
name = "clusterSkewX_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -150, xup=-150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew X", "label": "Cluster X Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewX efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [55]:
branch = "cscRechitClusterSkewY"
name = "clusterSkewY"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -150, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew Y", "label": "Cluster Y Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewY efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [56]:
branch = "cscRechitClusterSkewY"
name = "clusterSkewY_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -150, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew Y", "label": "Cluster Y Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewY efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [57]:
branch = "cscRechitClusterSkewZ"
name = "clusterSkewZ"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -150, xup=150)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew Z", "label": "Cluster Z Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [58]:
branch = "cscRechitClusterSkewZ"
name = "clusterSkewZ_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = -150, xup=150)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for entry in data_tree:
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(getattr(data_tree, branch)[i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = -5, xup=75)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(getattr(MC_tree, branch)[i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Cluster Skew Z", "label": "Cluster Z Skew [cm]", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [59]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s1_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s1_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s1_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 1", "label": "Station 1 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [60]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s1_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s1_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s1_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 1", "label": "Station 2 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [61]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s2"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s2_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s2_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 2", "label": "Station 2 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [62]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s2_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s2_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s2_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 2", "label": "Station 2 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [63]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s3"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s3_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s3_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 3", "label": "Station 3 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [64]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s3_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s3_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s3_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 3", "label": "Station 3 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [65]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_s4_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_s4_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_s4_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Station 4", "label": "Station 4 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [66]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw1"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw1_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw1_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 1", "label": "Ring 1 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [67]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw1_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw1_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw1_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 1", "label": "Ring 1 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [68]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw2"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw2_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw2_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 2", "label": "Ring 2 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [69]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw2_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data, [], True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw2_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC, [], True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw2_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 2", "label": "Ring 2 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [70]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw3"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw3_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw3_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 3", "label": "Ring 3 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  1.0


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [71]:
#branch = "cscRechitClusterSkewZ"
name = "clusterFrac_rw3_noCuts"
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=1.1)
mask = makeEventMask(events_data,[],True)
clusterIndex=-1
for evtNum, entry in enumerate(data_tree):
    for i in range(data_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            data.Fill(frac_rw3_data[evtNum][i])
data.SetLineColor(rt.kBlack)

sumOfWeights=0
mask = makeEventMask(events_MC,[],True)
#print(ak.count_nonzero(mask[~ak.is_none(mask)]))
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=1.1)
clusterIndex=-1
for evtNum, entry in enumerate(MC_tree):
    for i in range(MC_tree.nCscRechitClusters):
        clusterIndex+=1
        if mask[clusterIndex]:
            if MC_tree.cscRechitCluster_matchToMuon1[i]==1:
                probeIndex=0; tagIndex=1
            else:
                probeIndex=1; tagIndex=0
            weight = getattr(MC_tree, "weight")*getattr(MC_tree, "pileupWeight")*MC_SF_LooseID[evtNum][probeIndex]*MC_SF_LooseISO[evtNum][probeIndex]*MC_SF_TightID[evtNum][tagIndex]*MC_SF_TightID[evtNum][tagIndex]
            #weight = MC_SF_LooseID[evtNum][probeIndex]
            if MC_tree.lepPt[tagIndex]>26:
                weight=weight*MC_SF_HLT[evtNum][tagIndex]
            #print(weight)
            MC.Fill(frac_rw3_MC[evtNum][i], weight)
            sumOfWeights+=weight
print(branch + " efficiency  = ", num/sumOfWeights)        
MC.SetLineColor(rt.kRed)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Fraction of Hits in Ring 3", "label": "Ring 3 Hits/Total Hits", "scales": [sumOfWeights, data.GetEntries()], "logy": False}

cscRechitClusterSkewZ efficiency  =  0.03927046035048806


Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [72]:
for plot_type, plot_info_dict in plot_info.items():
    print(plot_type)
    #print(plot_info_dict)
    for boolScale in [True, False]:
        c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        if boolScale:
            scaleString = "_normalized"
            #c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2 hist','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        else:
            scaleString=""
        os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
        c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+"_updatedTNP.png")
        

ZMass
MuonVetoPt
MuonVetoPt_noCuts
DNNScore
DNNScore_noCuts
clusterSize
clusterSize_noCuts
clusterEta
clusterEta_noCuts
clusterPhi
clusterPhi_noCuts
clusterTime
clusterTime_noCuts
clusterTimeSpreadWeightedAll
clusterTimeSpreadWeightedAll_noCuts
clusterNStation10
clusterNStation10_noCuts
clusterXSpread
clusterXSpread_noCuts
clusterYSpread
clusterYSpread_noCuts
clusterZSpread
clusterZSpread_noCuts
clusterXYSpread
clusterRSpread
clusterRSpread_noCuts
clusterXYSpread_noCuts
clusterSkewX
clusterSkewX_noCuts
clusterSkewY
clusterSkewY_noCuts
clusterSkewZ
clusterSkewZ_noCuts
clusterFrac_s1_noCuts
clusterFrac_s2
clusterFrac_s2_noCuts
clusterFrac_s3
clusterFrac_s3_noCuts
clusterFrac_s4_noCuts
clusterFrac_rw1
clusterFrac_rw1_noCuts
clusterFrac_rw2
clusterFrac_rw2_noCuts
clusterFrac_rw3
clusterFrac_rw3_noCuts


Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange/ZMass_noCuts/ZMass_noCuts_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioZ Candidate Mass [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange/ZMass_noCuts/ZMass_noCuts_updatedTNP.png has been created
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange/MuonVetoPt/MuonVetoPt_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioMuon Veto Pt [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange/MuonVetoPt/MuonVetoPt_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioMuon Veto Pt [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_BothRings_DNNRange/MuonVetoPt_noCuts/MuonVetoPt_noCuts_normaliz

#### 